# Finding Steady states

Searching steady states in RTG model `rtgMTK` using `find_steady_states()`.

In [ ]:
using RetroSignalModel

paramsets = load_parameters()

In [ ]:
sol = find_steady_states(;S = t -> 0.0, params=paramsets[1])

**TODO**

Steady states at different levels of input signal `S`. (Use a loop).

## Saving results

> WIP